In [6]:
import os
import sys
import shutil
import random
import re
import gzip

if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import importlib
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import traci
import traci.constants as tc

# Add the "scripts" directory to sys.path
current_dir = os.path.dirname(os.path.abspath("__file__"))
scripts_dir = os.path.abspath(os.path.join(current_dir, 'scripts'))
sys.path.append(scripts_dir)
import vehParameters
import osmAPI as osm

FOLDER_NAME = "uah_map"

### Defining paths and setting up osm.sumocfg file

In [7]:
def add_xml_child(file_path, parent_tag, child_tag, child_value, replace=True):
    """
    Adds a new child parameter inside a specified parent tag in the XML configuration file.
    If the parent tag does not exist, it creates a new parent tag (<parameter>) with the child.
    It also checks if the child element already exists to prevent duplicates.

    Args:
        file_path (str): Path to the XML configuration file.
        parent_tag (str): The parent tag under which to add the child (e.g., 'input').
        child_tag (str): The child tag to add (e.g., 'additional-files').
        child_value (str): The value to set for the new child tag.
        replace (bool): If True, replaces the existing child tag with the new value.
                        If False, adds another child value.
    Returns:
        bool: True if the addition was successful, False otherwise.
    """
    try:
        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Find the parent element by tag
        parent_elem = root.find(parent_tag)
        if parent_elem is None:
            print(f"Parent tag '{parent_tag}' not found. Creating new parent tag.")
            parent_elem = ET.Element(parent_tag)
            root.append(parent_elem)
            print(f"Created new parent tag <{parent_tag}>.")

        # Check if the child element already exists inside the parent element
        existing_child = parent_elem.find(child_tag)
        if existing_child is not None: 
            
            if existing_child.get('value') == child_value:
                print(f"Child <{child_tag}> with value '{child_value}' already exists. Skipping addition.")
                return False
            else:
                if replace:
                    print(f"Child <{child_tag}> already exists. Updating value to '{child_value}'.")
                    existing_child.set('value', child_value)
                else:
                    if child_value in existing_child.get('value').split(', '):
                        print(f"Child <{child_tag}> with value '{child_value}' already exists. Skipping addition.")
                        return False
                    
                    print(f"Child <{child_tag}> already exists. Adding another child with value '{child_value}'.")
                    existing_child.set('value', f'{existing_child.get("value")}, {child_value}')
                
                tree.write(file_path, encoding="UTF-8", xml_declaration=True, method="xml")
                print("XML file updated successfully.")
                return True

        # Create the new child element and set its value
        new_child = ET.Element(child_tag)
        new_child.set('value', child_value)
        print(f"Created <{child_tag}> with value '{child_value}'.")

        # Add the new child to the parent element
        parent_elem.append(new_child)
        print(f"Added <{child_tag}> to <{parent_tag}>.")

        # Write the updated XML to the file
        tree.write(file_path, encoding="UTF-8", xml_declaration=True, method="xml")
        print("XML file updated successfully.")
        return True

    except ET.ParseError as e:
        print(f"XML Parsing error: {e}")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

In [8]:
NET_PATH = f'{FOLDER_NAME}/osm.net.xml'
PA_PATH = f'{FOLDER_NAME}/park.add.xml'
SUMOCFG_PATH = f'{FOLDER_NAME}/osm.sumocfg'
PA_REROUTER_PATH = f'{FOLDER_NAME}/pa_rerouter.xml'

# Setting up time to teleport as 
# -1 (never teleport): Vehicles will never teleport, but this could cause deadlock.
# >0 (seconds): Vehicles will teleport after the specified time if they are not able to reach their destination.
time_to_teleport = 100
add_xml_child(SUMOCFG_PATH, 'processing', 'time-to-teleport', f'{time_to_teleport}')

# Adding lateral resolution to use SubLane model
lateral_resolution = 0.8
add_xml_child(SUMOCFG_PATH, 'parameters', 'lateral-resolution', f'{lateral_resolution}')

# Setting up the timestep (default is 1 second), this does not affect the simulation speed but the resolution of the simulation. A smaller timestep will result in a more accurate simulation
step_length = 0.1
add_xml_child(SUMOCFG_PATH, 'time', 'step-length', f'{step_length}')

# Setting up the simulation duration. To simulate 24 hours, we set the end time to (3600 steps * 24 hours) / step_lenght.
end_time = (3600 * 24) / step_length
add_xml_child(SUMOCFG_PATH, 'time', 'end', f'{end_time}')

if not os.path.exists(NET_PATH):
    gz_path = f"{FOLDER_NAME}/osm.net.xml.gz"
    if os.path.exists(gz_path):
        with gzip.open(gz_path, 'rb') as f_in:
            with open(NET_PATH, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Extracted {gz_path} to {NET_PATH}")
    else:
        raise FileNotFoundError(f"{gz_path} does not exist.")
    
net = sumolib.net.readNet(NET_PATH)

if not os.path.exists(PA_PATH):
    print(f"{PA_PATH} does not exist. No using any parking areas.")
else:
    parkingAreas = list(sumolib.output.parse(PA_PATH, "parkingArea"))
    # Setting up additional files
    add_xml_child(SUMOCFG_PATH, 'input', 'additional-files', "park.add.xml, pa_rerouter.xml")

net_offset = net.getLocationOffset() # Necessary to convert from CARLA to SUMO coordinates if needed

Child <time-to-teleport> with value '100' already exists. Skipping addition.
Child <lateral-resolution> with value '0.8' already exists. Skipping addition.
Child <step-length> already exists. Updating value to '0.1'.
XML file updated successfully.
Child <end> already exists. Updating value to '864000.0'.
XML file updated successfully.
uah_map/park.add.xml does not exist. No using any parking areas.


In [9]:
def flush_print():
    sys.stdout.write("\r")  # Move the cursor to the beginning of the line
    sys.stdout.write(" " * 50)  # Overwrite with spaces to clear the line
    sys.stdout.write("\r")  # Move back to the beginning again

In [10]:
def has_parking_spot(lanes, parkingAreas):
    # Example of parkingArea:
    # <parkingArea id="pa-1046248579#0" lane="-1046248579#0_0" roadsideCapacity="94" length="5.00"/>
    # Returns parkingArea id if there is a parking spot in the lane
    lane_ids = [lane.getID() for lane in lanes]
    for park in parkingAreas:
        if park.lane in lane_ids:
            return park.id
    return False

In [11]:
def getClosestEdges(lat, lon, radius, maxEdges=10, convert_toXY=True):

    # Gets the 10 closest edges to the given lat, lon
    if convert_toXY:
        x, y = net.convertLonLat2XY(lon, lat)
    else:
        x, y = lon, lat
    edges = net.getNeighboringEdges(x, y, radius)
    closestEdges = []
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])

        ## Checking if the edge found can be used by passenger car
        for dist, edge in distanceAndEdges:
            if edge.allows('passenger'):
                closestEdges.append(edge)

    if len(edges) == 0:
        print(f'No edges found for {lat}, {lon}. Perhaps location is not inside the network or there are no viable roads inside the radius.')
        return None
    
    return closestEdges

In [12]:
def getParkingSpot(lat, lon, radius, parkingAreas):
    # Get the parking spot closest to the given lat, lon
    # Used to set stops for the vehicles

    edges = getClosestEdges(lat, lon, radius)
    # Look for parking spots
    for i in range(len(edges)):
        parking_spot = has_parking_spot(edges[i].getLanes(), parkingAreas)
        if parking_spot:
            return parking_spot
    print(f"No parking spot found close to {lat}, {lon}. Perhaps decrease the radius?")
    return None

In [13]:
def getPath(location_time_list, parkingAreas, steps_per_stop, radius = 100, use_carla_routine=False):
    # All that is needed to create the trip are the stops (parking areas) and the start and end edges.
    # The duarouter is responsible for finding the path between the edges going through the stops.
    # Here, we get the edges and stops that are going to be sent to LLAMA to create the trip.

    # 'coordinates' is a list of tuples with the latitude and longitude of the points of interest, for example IC, FEEC, IC means that
    # the vehicle will start from IC, stop at a parking lot close to FEEC, and then back to IC.
    # The first and last coordinates should be edges and the others should be parking spots.
    # `steps_per_stop` is the number of simulation steps that the vehicle will stay at each stop.

    # Departure for 7 is 0, 8 is 100, 9 is 200 and so on
    stop_durations = []
    departures = list(location_time_list.keys())
    stop_durations.append(-1) # Indicates this is an edge and not a parking spot

    path = []
    coords = [location_time_list[k]['coords'] for k in location_time_list.keys()]
    
    if use_carla_routine: 
        convert_toXY = False # No need to convert from lat, lon to XY
        
    home = getClosestEdges(*coords[0], radius, convert_toXY=convert_toXY)[0].getID()
    path.append(home)
        
    for i in range(1, len(coords)-1):

        stop_durations.append(steps_per_stop * (departures[i + 1] - departures[i]))
        if use_carla_routine: # When using the carla routine, we don't have parking spots
            ps = getClosestEdges(*coords[i], radius, convert_toXY=convert_toXY)[0].getID()
        else:
            ps = getParkingSpot(*coords[i], radius, parkingAreas)

        if ps is not None:
            path.append(ps)
        else:
            print(f"Could not find parking spot for {coords[i]}")
            raise Exception(f"Could not find parking spot for {coords[i]}")

    path.append(home)
    stop_durations.append(-1)
    
    return path, stop_durations

In [14]:
def clean_response(institutes):
    cleaned_institutes = []
    for institute in institutes:
        # Remove parenthesis and everything after it
        institute = re.sub(r'\(.*', '', institute).strip()
        
        # Split by hyphen and take the longest slice
        parts = institute.split('-')
        longest_part = max(parts, key=len).strip()
        # Split by '/' and add both parts to the cleaned_institutes list
        if '/' in longest_part:
            parts = longest_part.split('/')
            for part in parts:
                part = part.strip()
                if part not in cleaned_institutes:
                    cleaned_institutes.append(part)
        else:
            if longest_part not in cleaned_institutes:
                cleaned_institutes.append(longest_part)
        
    return cleaned_institutes

In [15]:
def getCoords(trip, sulfixo, institutes, start_radius, step_radius, limit_radius, uni_center_lat, uni_center_lon, n_options = 3, restaurants = None):
    # Returns a dictionary with the latitude and longitude of the locations of interest as well as the name of the location
    # The suffix is the name of the state, city and neighborhood that will be added to the end of each location to improve the search
    # 'start_radius' is the initial radius of the search, 'step_radius' is the amount that will be added to the radius if the location is not found and 'limit_radius' is the maximum radius that will be used. After that, the student will choose not to leave the place he is at.
    # 'n_options' is the number of options of places we ideally want to find to choose from. This only applies while the limit_radius is not exceeded
    
    coords = {} # Coordinates for every place the student will visit
    names = {}
    importlib.reload(osm)
    for i in range(len(trip)):
        local = trip[f'{i + 7}']['location']
        local_comp = local + ", " + sulfixo

        if local in coords.keys(): # If the location is already in the dictionary, use the coordinates from there
            lat, lon = coords[local]
            name = names[local]
            continue
        
        if local == 'home': # If the location is home, use the home coordinates
            coords['home'] = getHome()
            names['home'] = 'home'
            continue
        
        elif (local in institutes) or (restaurants and local in restaurants): # If the location is an institute, use the coordinates from the API
                result = osm.geocode_address(local_comp)
                
                if not result:
                    raise Exception(f"Could not get coordinates for {local}, maybe its name is not correct")

                name = local
                lat, lon = result[0]['latitude'], result[0]['longitude']
                print(f"\033[1mFound {local} at {lat}, {lon}.\033[0m")

        
        else: # If the location is not an institute, we have to search for it in the OSM API
            found = False
            print(f"Looking for {local}...", end='', flush=True)

            # Storing the previous location to use as a reference for the next search
            names[local] = local
            result = osm.find_nearby_building(uni_center_lat, uni_center_lon, local, radius=start_radius)
            expanded = start_radius
            if len(result) > 0: # Found at least one option
                found = True

            # If the location is not found or there are less optios than expected, expand the search radius
            while len(result) == 0 or len(result) < n_options:
                expanded += step_radius

                if expanded > limit_radius:
                    break

                result = osm.find_nearby_building(uni_center_lat, uni_center_lon, local, radius=expanded)
                if found == False and (len(result) > 0):
                    found = True # Found at least one option, but will keep looking for more until the limit is reached

            if found == False:
                flush_print()
                print(f"Could not find {local} in a radius of {limit_radius} meters. The student will not leave the place he is currently at.")

            else:
                random.shuffle(result) # Randomize the results to avoid always getting the absolute closest building
                lat, lon = result[0]['latitude'], result[0]['longitude']
                name = result[0]['name']
                flush_print()
                print(f"Found {len(result)} options for {local}: {[x['name'] for x in result]}")
                print(f"\033[1m{local} picked: {result[0]['name']} at {lat}, {lon}.\033[0m")

        
        coords[f'{local}'] = (lat, lon)
        names[f'{local}'] = name
        
    return coords, names

In [16]:
def coordsToTrip(trip, coords, names = None):
    location_time = {}
    # The first location is always home
    location_time[7] = {}
    location_time[7]['location'] = 'home'
    location_time[7]['coords'] = coords['home']
    location_time[7]['name'] = 'home'
    last = coords['home']
    
    for j in range(1, len(trip)):
        location = trip[f'{j + 7}']['location']
        location_coords = coords[location]
        if names:
            location_names = names[location]
        else:
            location_names = location

        if location_coords != last:
            # If the coordinates are the same as the last one, skip this location
            location_time[j + 7] = {}
            location_time[j + 7]['location'] = location
            location_time[j + 7]['coords'] = location_coords
            location_time[j + 7]['name'] = location_names
            last = location_coords

    return location_time

In [17]:
def pathToXML(path, vehicleID, veh_type, departure_time, stop_durations, no_parking=False):
    # Converts the path to the XML format that LLAMA understands
    xml = f'<trip id="{vehicleID}" type="{veh_type}" depart="{departure_time}" from="{path[0]}" to="{path[-1]}">\n'
    for i in range(1, len(path)-1):
        if no_parking:
            xml += f'\t<stop edge="{path[i]}" duration="{stop_durations[i]}"/>\n'
        else:
            xml += f'\t<stop parkingArea="{path[i]}" duration="{stop_durations[i]}"/>\n'

    xml += '</trip>'
    return xml

In [18]:
def add_missing_vtypes(routes_file, types_file, output_file):
    # Parse the XML files
    routes_tree = ET.parse(routes_file)
    routes_root = routes_tree.getroot()
    types_tree = ET.parse(types_file)
    types_root = types_tree.getroot()
    
    # Extract existing vType IDs in routes_file
    existing_vtypes = {vtype.get('id') for vtype in routes_root.findall('vType')}
    
    # Find vTypes in types_file that are not in routes_file
    for vtype in types_root.findall(".//vType"):
        if vtype.get('id') not in existing_vtypes:
            routes_root.insert(0, vtype)
    
    # Write the updated routes file
    routes_tree.write(output_file, encoding='utf-8', xml_declaration=True)
    print(f"Updated ROUTES_FILE saved as {output_file}")

Here we will NOT USE CAR-FOLLOWING MODEL PARAMETERS because there are no other vehiles in the simulation. Details of the parameters are documented on the paper

In [19]:
styles = ["normal", "aggressive"]
param_dict = {
    'maxSpeed': {
        'normal': 50 / 3.6, # m/s
        'aggressive': 70 / 3.6,
    },
    'accel': { # Default value for SUMO passenger car is 2.6
        'normal': 2.5,
        'aggressive': 3.0,
    },
    'decel': { # Default value for SUMO is 4.5
        'normal': 4.5,
        'aggressive': 5.0,
    },
    'speedFactor': { # Slightly higher for aggressive drivers
        'normal': 1.0,
        'aggressive': 1.2,
    },
    'emergencyDecel': {
        'normal': 9.0,
        'aggressive': 10.0,
    },
    'vClass': {
        'normal': 'passenger',
        'aggressive': 'passenger',
    }
}

In [20]:
importlib.reload(vehParameters)
# Parsing the vehicles to the XML format and writing to vTypesDistribution.xml
vehParameters.parseVehicleTypesXML(param_dict, styles, FOLDER_NAME, car_follow_model="EIDM", lc_model="SL2015")

'<root>\n<vType id="normal" carFollowModel="EIDM" laneChangeModel="SL2015" maxSpeed=\'13.88888888888889\' accel=\'2.5\' decel=\'4.5\' speedFactor=\'1.0\' emergencyDecel=\'9.0\' vClass=\'passenger\' >\n\t<param key="device.rerouting.probability" value="1.0"/>\n\t<param key="device.rerouting.adaptation-steps" value="18"/>\n\t<param key="device.rerouting.adaptation-interval" value="10"/>\n</vType>\n<vType id="aggressive" carFollowModel="EIDM" laneChangeModel="SL2015" maxSpeed=\'19.444444444444443\' accel=\'3.0\' decel=\'5.0\' speedFactor=\'1.2\' emergencyDecel=\'10.0\' vClass=\'passenger\' >\n\t<param key="device.rerouting.probability" value="1.0"/>\n\t<param key="device.rerouting.adaptation-steps" value="18"/>\n\t<param key="device.rerouting.adaptation-interval" value="10"/>\n</vType>\n</root>\n'

The desired edges need to be set. Their IDs are determined using **netedit uah_map/osm.net.xml** and looking for the desired edge in the map.

In [21]:
start_edge, end_edge = 28308066, -125136020 # Got edges from the net file
list_of_ids = []

os.makedirs(f"{FOLDER_NAME}/experiment1", exist_ok=True)
for idx, style in enumerate(styles):
    id = f'veh{idx}_{style}'
    xml = pathToXML([start_edge, end_edge], f'veh{idx}_{style}', style, 0, [0, 0], no_parking=True) # Creates a test trip to check if the vehicle is working

    # Write the XML to a file
    exp1_trip_file = f"{FOLDER_NAME}/experiment1/{id}.rou.xml"
    with open(exp1_trip_file, "w") as f:
        f.write(f'<routes>\n{xml}\n</routes>')
    add_missing_vtypes(exp1_trip_file, f"{FOLDER_NAME}/vTypesDistribution.xml", exp1_trip_file)

    list_of_ids.append(id)

# Writing the list of ids to a file
with open(f"{FOLDER_NAME}/experiment1/vehicle_ids.txt", "w") as f:
    for veh_id in list_of_ids:
        f.write(f"{veh_id}\n")

Updated ROUTES_FILE saved as uah_map/experiment1/veh0_normal.rou.xml
Updated ROUTES_FILE saved as uah_map/experiment1/veh1_aggressive.rou.xml


Now the routes for the vehicles are saved to the experiment1 folder and we have to run the simulation

## Simulation

In [22]:
def get_all_variables(vehIDs, personIDs, routine=None, delta_time=0.05, end_hours=24, useGui=False, convertGeo=True, freeze_traffic_lights=False):
    """
    Function to get all variables from the simulation.
    :param vehIDs: List of vehicle IDs to subscribe to.
    :param personIDs: List of person IDs to subscribe to.
    :param routine: Dictionary with the routine of the vehicles.
    :param useGui: Boolean to use the GUI or not.
    :param convertGeo: Boolean to convert the coordinates from sumo to lat/lon (does not work when using maps from CARLA).
    :param freeze_traffic_lights: If true, traffic lights are green all the time.
    :return: Dictionary with the variables.
    """

    if useGui:
        traci.start(["sumo-gui", "-c", f"{FOLDER_NAME}/osm.sumocfg"])
    else:
        traci.start(["sumo", "-c", f"{FOLDER_NAME}/osm.sumocfg"])
    v_variables = {}
    p_variables = {}

    if end_hours != 0:
        end_time = f'{end_hours * 60 * (60/delta_time)}' # 24 hours
    else:
        end_time = ''
    
    add_xml_child(f'{FOLDER_NAME}/osm.sumocfg', 'time', 'step-length', f'{delta_time}', replace=True)
    add_xml_child(f'{FOLDER_NAME}/osm.sumocfg', 'time', 'end', end_time, replace=True)

    # Get all traffic light IDs
    if freeze_traffic_lights:
        tls_ids = traci.trafficlight.getIDList()

        # Set all traffic lights to constant green
        for tls_id in tls_ids:
            # Get logic info
            logic = traci.trafficlight.getCompleteRedYellowGreenDefinition(tls_id)[0]

            # Create a new green-only phase
            green_state = 'G' * len(logic.phases[0].state)
            green_phase = traci.trafficlight.Phase(duration=9999999, state=green_state)

            # Replace the logic with only this green phase
            new_logic = traci.trafficlight.Logic(
                logic.programID, logic.type, logic.currentPhaseIndex, [green_phase]
            )
            traci.trafficlight.setCompleteRedYellowGreenDefinition(tls_id, new_logic)
    
    # To understand when a car has changed its destionation, we need to know it parked
    if routine:
        status = {}
        for id in routine.keys():
            status[id] = {}
            status[id]['parked'] = False # True if the vehicle is parked
            status[id]['count'] = 0 # Count of number of times a vehicle has parked
            status[id]['stops'] = list(routine[id]['name']) # List of stops for the vehicle
            status[id]['desc'] = f'<home-{status[id]["stops"][1]}>' # Description of the vehicle's status

    time = 0
    while traci.simulation.getMinExpectedNumber() > 0:

        for veh_id in (set(traci.simulation.getDepartedIDList()) & set(vehIDs)): # Subscribe to vehicles that have just departed
            print(f"Vehicle {veh_id} has departed")
            traci.vehicle.subscribe(veh_id, [tc.VAR_POSITION, tc.VAR_SPEED, tc.VAR_ACCELERATION, tc.VAR_ANGLE])
        
        for veh_id in (set(traci.vehicle.getIDList()) & set(vehIDs)): # Checking status of vehicles that are already in the simulation
            if routine:
                is_parked = traci.vehicle.isStoppedParking(veh_id)
                if not status[veh_id]['parked'] and is_parked:  # Vehicle was not parked and now is parked
                    print(f"Vehicle {veh_id} is parked.")
                    status[veh_id]['parked'] = True
                    status[veh_id]['count'] += 1
                    current_location = status[veh_id]["stops"][status[veh_id]["count"]]
                    next_location = status[veh_id]["stops"][status[veh_id]["count"] + 1] if status[veh_id]["count"] + 1 < len(status[veh_id]["stops"]) else None
                    status[veh_id]['desc'] = f'<{current_location}-{current_location}>'

                elif status[veh_id]['parked'] and not is_parked:  # Vehicle was parked and now is not parked
                    status[veh_id]['parked'] = False
                    current_location = status[veh_id]["stops"][status[veh_id]["count"]]
                    next_location = status[veh_id]["stops"][status[veh_id]["count"] + 1] if status[veh_id]["count"] + 1 < len(status[veh_id]["stops"]) else None
                    status[veh_id]['desc'] = f'<{current_location}-{next_location}>'

        for veh_id in (set(traci.simulation.getArrivedIDList()) & set(vehIDs)): # Vehicles that finished their route
            # traci.vehicle.unsubscribe(veh_id)
            if routine:
                status[veh_id]['count'] += 1
                current_location = status[veh_id]["stops"][status[veh_id]["count"]]
                status[veh_id]['desc'] = f'<{current_location}-{current_location}>'

        results = traci.vehicle.getAllSubscriptionResults().copy()

        for veh_id in results.keys():
            # Converting from x, y sumo coordinates to lat, lon
            if convertGeo:
                x, y = results[veh_id][tc.VAR_POSITION]
                lon, lat = traci.simulation.convertGeo(x, y, fromGeo=False)
                results[veh_id]['longitude'] = lon
                results[veh_id]['latitude'] = lat
                
            if routine:
                results[veh_id]['desc'] = status[veh_id]['desc']

        v_variables[time] = results
        
        time += delta_time
        traci.simulationStep()
        
    traci.close()
    return v_variables, p_variables

In [23]:
def save_data(veh_variables, data_folder_name, vehIDs, delta_time, file_names_postfix = '', new_dir = False, type_ids_random = None, verify=True, type_ids_routine=None, use_desc=True, use_lat_lon=True, speed_threshold=6, acc_threshold=6, derivative_threshold=3):

    if new_dir:
        if not os.path.exists(f'{FOLDER_NAME}/{data_folder_name}'):
            os.mkdir(f'{FOLDER_NAME}/{data_folder_name}')
        else:
            shutil.rmtree(f'{FOLDER_NAME}/{data_folder_name}')
            os.mkdir(f'{FOLDER_NAME}/{data_folder_name}')

    for timestep, data in veh_variables.items():
        for vehID, vehData in data.items():

            if use_lat_lon:
                columns = 'timestamp,latitude,longitude,speed,speed_x,speed_y,acceleration,acceleration_x,acceleration_y,angle,acc_diff,gyroscope_z'
            else:
                columns = 'timestamp,x_pos,y_pos,speed,speed_x,speed_y,acc,acc_x,acc_y,angle,acc_diff,gyro_z'

            path = f'{FOLDER_NAME}/{data_folder_name}/{vehID}{file_names_postfix}.csv'
            nolabel_path = f'{FOLDER_NAME}/{data_folder_name}/{vehID}{file_names_postfix}_nolabel.csv'
            if use_desc:
                if not os.path.exists(path):

                    with open(path, 'w') as f:
                        f.write(f'{columns},desc\n')
            
            if not os.path.exists(nolabel_path):
                with open(nolabel_path, 'w') as f:
                    f.write(f'{columns}\n')
                    
            write_speed = vehData[SPEED]
            write_angle = vehData[ANGLE]
            write_acc = vehData[ACCELERATION]
            write_x = vehData[POSITION][0]
            write_y = vehData[POSITION][1]

            if verify:
                derivative_threshold
                try: 
                    derivative_speed = (veh_variables[timestep][vehID][SPEED] - veh_variables[timestep-derivative_threshold][vehID][SPEED]) / derivative_threshold
                    derivative_acceleration = (veh_variables[timestep][vehID][ACCELERATION] - veh_variables[timestep-derivative_threshold][vehID][ACCELERATION]) / derivative_threshold
                except:
                    derivative_speed = 0
                    derivative_acceleration = 0
                    
                # Making verification to ensure there are no outliers
                if derivative_speed > speed_threshold or derivative_speed < -speed_threshold:
                    last_speed = veh_variables[timestep-delta_time][vehID][SPEED]
                    print(f'Vehicle {vehID} at timestep {timestep} had a speed of {vehData[SPEED]}, it was changed to {last_speed}')
                    veh_variables[timestep][vehID][SPEED] = last_speed
                    write_speed = last_speed

                if derivative_acceleration > acc_threshold or derivative_acceleration < -acc_threshold:
                    last_acc = veh_variables[timestep - delta_time][vehID][ACCELERATION]
                    print(f'Vehicle {vehID} at timestep {timestep} had an acceleration of {vehData[ACCELERATION]}, it was changed to {last_acc}')
                    veh_variables[timestep][vehID][ACCELERATION] = last_acc
                    write_acc = last_acc
                
                if vehData[ANGLE] < 0 or vehData[ANGLE] > 360:
                    last_angle = veh_variables[timestep - delta_time][vehID][ANGLE]
                    print(f'Vehicle {vehID} at timestep {timestep} had an angle of {vehData[ANGLE]}, it was changed to {last_angle}')
                    veh_variables[timestep][vehID][ANGLE] = last_angle
                    write_angle = last_angle

            if write_speed < -100: # Invalid values
                write_speed = 0
                print(f'Vehicle {vehID} at timestep {timestep} had a INVALID speed of {vehData[SPEED]}, it was changed to 0')
            if write_acc < -100 or write_acc > 100: # Invalid values
                write_acc = 0
                print(f'Vehicle {vehID} at timestep {timestep} had a INVALID acceleration of {vehData[ACCELERATION]}, it was changed to 0')

            # Calculating the decomposed acceleration and speed
            write_speed_x = write_speed * np.cos(np.radians(write_angle))
            write_speed_y = write_speed * np.sin(np.radians(write_angle))
            write_acc_x = write_acc * np.cos(np.radians(write_angle))
            write_acc_y = write_acc * np.sin(np.radians(write_angle))
            
            try:
                acc_diff = np.abs((veh_variables[timestep][vehID][ACCELERATION] -
                                veh_variables[timestep - delta_time][vehID][ACCELERATION]) / delta_time)

                current_angle = np.radians(veh_variables[timestep][vehID][ANGLE])
                previous_angle = np.radians(veh_variables[timestep - delta_time][vehID][ANGLE])

                # Proper angle wrapping to handle transitions like 359° -> 0°
                angle_diff = np.arctan2(np.sin(current_angle - previous_angle),
                                        np.cos(current_angle - previous_angle))

                gyroscope_z = angle_diff / delta_time  # radians/second
            except:
                acc_diff = np.abs(write_acc)
                gyroscope_z = 0.0

            if use_lat_lon:
                line = f'{timestep},{vehData["latitude"]},{vehData["longitude"]},{write_speed},{write_speed_x},{write_speed_y},{write_acc},{write_acc_x},{write_acc_y},{write_angle},{acc_diff},{gyroscope_z}'
            else:
                line = f'{timestep},{write_x},{write_y},{write_speed},{write_speed_x},{write_speed_y},{write_acc},{write_acc_x},{write_acc_y},{write_angle},{acc_diff},{gyroscope_z}'

            if use_desc:
                with open(path, 'a') as f:
                    f.write(f'{line},{vehData["desc"]}\n')
            
            with open(nolabel_path, 'a') as f:
                f.write(f'{line}\n')
        
    # Creating the labels file
    if use_desc:
        with open(f'{FOLDER_NAME}/{data_folder_name}/labels.csv', 'w') as f:
            f.write('ID,Type\n')
            found = False
            for i in range(len(vehIDs)):
            
                if type_ids_random and not found:
                    for key, value in type_ids_random.items():
                        if vehIDs[i] in str(value):
                            f.write(f'{vehIDs[i]},{key}\n')
                            found = True
                            break
                    
                if type_ids_routine and not found:
                    for key, value in type_ids_routine.items():
                        if vehIDs[i] in str(value):
                            f.write(f'{vehIDs[i]},{key}\n')
                            break

In [24]:
sumoBinary = "/usr/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "osm.sumocfg"]

# Code of each variable to subscribe:
SPEED = 64
POSITION = 66
ACCELERATION = 114
ANGLE = 67
DEPATURE = 58

In [25]:
output_data_folder = "data_experiment1_sumo"

In [26]:
experiment1_files = os.listdir(f'{FOLDER_NAME}/experiment1')
delta_time = 0.01 # This delta time has to be the same as the one used in CARLA
end_time = 0 # 0 means no end time, the simulation will run until all vehicles finish their route
with open(f'{FOLDER_NAME}/experiment1/vehicle_ids.txt', 'r') as f:
    vehIDs = f.read().splitlines()


if os.path.exists(f'{FOLDER_NAME}/{output_data_folder}'):
    shutil.rmtree(f'{FOLDER_NAME}/{output_data_folder}')
os.makedirs(f'{FOLDER_NAME}/{output_data_folder}', exist_ok=True)

for i in range(len(vehIDs)):

    add_xml_child(f'{FOLDER_NAME}/osm.sumocfg', 'input', 'route-files', f'experiment1/{vehIDs[i]}.rou.xml', replace=True)
    personIDs = []

    try:
        veh_variables, ped_variables = get_all_variables(vehIDs, personIDs, delta_time=delta_time, routine=None, end_hours=end_time, useGui=False, convertGeo=False, freeze_traffic_lights=True) # Running the simulation
        save_data(veh_variables, output_data_folder, [vehIDs[i]], delta_time, new_dir=False, verify=True, use_lat_lon=False, use_desc=False)
        
    except KeyboardInterrupt as e:
        traci.close() # Close the simulation if there is an error
        print(f"An error occurred: {e}")

Child <route-files> already exists. Updating value to 'experiment1/veh0_normal.rou.xml'.
XML file updated successfully.
 Retrying in 1 seconds
***Starting server on port 48833 ***
Loading net-file from 'uah_map/osm.net.xml.gz' ... done (27ms).
Loading route-files incrementally from 'uah_map/experiment1/veh0_normal.rou.xml'
Loading done.
Simulation version 1.23.1 started with time: 0.00.
Child <step-length> already exists. Updating value to '0.01'.
XML file updated successfully.
Child <end> already exists. Updating value to ''.
XML file updated successfully.
Vehicle veh0_normal has departed
Simulation ended at time: 486.40.
Reason: TraCI requested termination.
Performance:
 Duration: 1.64s
 TraCI-Duration: 1.52s
 Real time factor: 296.766
 UPS: 2967.053081
Vehicles:
 Inserted: 1
 Running: 0
 Waiting: 0
Statistics (avg of 1):
 RouteLength: 6664.31
 Speed: 13.70
 Duration: 486.30
 WaitingTime: 0.30
 TimeLoss: 6.39
 DepartDelay: 0.00
DijkstraRouter answered 1 queries and explored 17.00 edg